# Preparing Snowflake
This note book will guide you through the required steps thats need to prepare your Snowflake account to deploy the demo [Streamlit ML App](https://github.com/kameshsampath/st-ml-app)

Typically we will

- [x] Create Schemas and Tables
- [x] Create a external stage to S3
- [x] Load Penguins Dataset on the table
- [x] Create Snowpark Contiainer Services(SPCS) objects

## Schemas and Stages

In the next cell we will create the schemas and stages that will be used in this tutorial.

### Schemas

|Schema | Use|
|------- |----------------|
| apps | Will hold all applications e.g. Streamlit|
| data | Will hold all data tables  |
| stages | All  stages |
| file_formats | All  file formats that will be used during data load |



In [ ]:
-- data schema
CREATE SCHEMA IF NOT EXISTS DATA;
-- create schema to hold all stages
CREATE SCHEMA IF NOT EXISTS STAGES;
-- create schema to hold all file formats
CREATE SCHEMA IF NOT EXISTS FILE_FORMATS;
-- apps to hold all streamlit apps
CREATE SCHEMA IF NOT EXISTS APPS;


### Stages and File Format

We will create stage named `stages.st_ml_app_penguins` which will point to an s3 bucket `s3://sfquickstarts/misc` and it will use the file file format `file_formats.csv` to parse and laod CSV files.

In [ ]:
-- add an external stage to a s3 bucket
CREATE STAGE IF NOT EXISTS STAGES.ST_ML_APP_PENGUINS
  URL='s3://sfquickstarts/misc';

-- default CSV file format and allow values to quoted by "
CREATE FILE FORMAT IF NOT EXISTS FILE_FORMATS.CSV
  TYPE='CSV'
  SKIP_HEADER=1
  FIELD_OPTIONALLY_ENCLOSED_BY = '"';

## Load 

As part of next step let us load the data on `data.penguins` table using the file from `@stages.st_ml_app_penguins/penguins_cleaned.csv`

In [ ]:
-- Create table to hold penguins data
CREATE OR ALTER TABLE DATA.PENGUINS(
   SPECIES STRING NOT NULL,
   ISLAND STRING NOT NULL,
   BILL_LENGTH_MM NUMBER NOT NULL,
   BILL_DEPTH_MM NUMBER NOT NULL,
   FLIPPER_LENGTH_MM NUMBER NOT NULL,
   BODY_MASS_G NUMBER NOT NULL,
   SEX STRING NOT NULL
);

-- Load the data from penguins_cleaned.csv
COPY INTO DATA.PENGUINS
FROM @stages.st_ml_app_penguins/penguins_cleaned.csv
FILE_FORMAT=(FORMAT_NAME='FILE_FORMATS.CSV');

Let us select and verify the data,

In [ ]:
from snowflake.snowpark.context import get_active_session

session = get_active_session()
df = session.table('st_ml_app.data.penguins')
df.show(10)

## Telemetry Settings
In the following steps, we will set up Telemetry Events (logs/traces) at the database level. While Snowflake defaults to storing events in `SNOWFLAKE.TELEMETRY.EVENTS`, for this demo we will configure event collection at the database level.

Create the event table at the database level and set it as the default Events table for the database.

In [ ]:
-- create schema
CREATE SCHEMA IF NOT EXISTS ST_ML_APP.telemetry;
-- create event table 
CREATE EVENT TABLE IF NOT EXISTS ST_ML_APP.telemetry.events;
-- set to new event table
ALTER DATABASE ST_ML_APP SET EVENT_TABLE = ST_ML_APP.telemetry.events;

Create the event table at the database level and set it as the default Events table for the database.

In [ ]:
-- set log, trace and metrtic levels
ALTER DATABASE ST_ML_APP SET LOG_LEVEL = DEBUG;
ALTER DATABASE ST_ML_APP SET TRACE_LEVEL = ALWAYS;
ALTER DATABASE ST_ML_APP SET METRIC_LEVEL = ALL;

> **IMPORTANT:**
>
> The following sections are required only if you are doing SPCS sections of the lab. 
> SPCS is not available for enterprise trials.

## Snowpark Container Services(SPCS)

In [ ]:
SELECT current_role() as current_role

In [ ]:
SELECT current_database() as current_database

In [ ]:
SELECT current_user() as current_user

In [ ]:
__current_role=sql_current_user_role.to_pandas().iloc[0]['CURRENT_ROLE']
__current_db=sql_current_database.to_pandas().iloc[0]['CURRENT_DATABASE']
__current_user=sql_current_user.to_pandas().iloc[0]['CURRENT_USER']
__spcs_st_ml_app_role='st_ml_app'
__spcs_role_name=__current_role
__spcs_images_schema='images'
__spcs_wh_name='st_ml_app_spcs_wh_s'
__spcs_compute_pool='st_ml_app_xs'


In [ ]:
-- Grant ownership on the DB to ACCOUNTADMIN as we will be creating few other objects which might require higher privileges
-- If you have created all objects as ACCOUNTADMIN then you can skip this step i.e `__current_role` is ACCOUNTADMIN
GRANT OWNERSHIP ON DATABASE {{__current_db}} TO ROLE ACCOUNTADMIN COPY CURRENT GRANTS;

USE ROLE ACCOUNTADMIN;

-- Role that will be used to create services
CREATE ROLE IF NOT EXISTS {{__spcs_st_ml_app_role}};

-- grant {{__spcs_st_ml_app_role}} role to current user
GRANT ROLE {{__spcs_st_ml_app_role}} TO USER {{__current_user}};

-- Grant ownership on the DB to {{__spcs_st_ml_app_role}}
GRANT OWNERSHIP ON DATABASE {{__current_db}} TO ROLE {{__spcs_st_ml_app_role}} COPY CURRENT GRANTS;

-- use {{__spcs_st_ml_app_role}} to create the data schema
USE ROLE {{__spcs_st_ml_app_role}};

-- data_schema will house the image repositories
CREATE SCHEMA IF NOT EXISTS {{__current_db}}.{{__spcs_images_schema}};
GRANT OWNERSHIP ON SCHEMA {{__current_db}}.{{__spcs_images_schema}} TO ROLE {{__spcs_st_ml_app_role}} ;
-- grant __spcs_st_ml_app_role to own other schemas too to allow the Streamlit app within
-- the container to access the tables
GRANT OWNERSHIP ON SCHEMA {{__current_db}}.DATA TO ROLE {{__spcs_st_ml_app_role}} ;
GRANT OWNERSHIP ON TABLE {{__current_db}}.DATA.PENGUINS TO ROLE {{__spcs_st_ml_app_role}} ;

-- Switch back to accountadmin for rest of the tasks
USE ROLE ACCOUNTADMIN;

-- Create warehouse to be used for queries from the service
CREATE WAREHOUSE IF NOT EXISTS {{__spcs_wh_name}} WITH
  WAREHOUSE_SIZE='X-SMALL'
  -- disable auto start
  INITIALLY_SUSPENDED=TRUE
  -- auto suspend in two mins
  AUTO_SUSPEND=120;

-- grants on warehouse to {{__spcs_st_ml_app_role}}
GRANT USAGE ON WAREHOUSE {{__spcs_wh_name}} TO ROLE {{__spcs_st_ml_app_role}};

-- allow endpoint binding to role 
GRANT BIND SERVICE ENDPOINT ON ACCOUNT TO ROLE {{__spcs_st_ml_app_role}};

-- allow role to use and monitor compute pool
GRANT USAGE, MONITOR ON COMPUTE POOL {{__spcs_compute_pool}} TO ROLE {{__spcs_st_ml_app_role}};

## Cleanup
SPCS has compute associated with it, run the following cell to clean the Snowflake resources created as part of the demo.

In [ ]:
-- drop the service
DROP SERVICE IF EXISTS {{__current_db}}.{{__spcs_images_schema}}.ST_ML_APP;

USE ROLE ACCOUNTADMIN;

-- gracefully stop and delete all services running on this compute pool
ALTER COMPUTE POOL {{__spcs_compute_pool}} STOP ALL;

DROP COMPUTE POOL IF EXISTS {{__spcs_compute_pool}};

-- drop role 
DROP ROLE IF EXISTS {{__spcs_st_ml_app_role}};

-- suspend warehouse 
ALTER WAREHOUSE {{__spcs_wh_name}} SUSPEND;